In [1]:
import os
import sys
import yaml
import wandb
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn.functional as F
import time
from torch.utils.data import DataLoader

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
%pip install wandb -qU
%matplotlib inline

# Get the current working directory
notebook_dir = notebook_dir = os.path.dirname(os.path.abspath("__file__"))  
project_dir = os.path.abspath(os.path.join(notebook_dir, '..')) 
if project_dir not in sys.path:
    sys.path.append(project_dir)

from binary_classifier import (get_transforms, load_data, split_data, set_seeds, 
                 verify_splits, verify_data, plot_species_grid,
                 verify_loader_transforms)
from binary_classifier.data_utils import ImagesDataset
from binary_classifier.models import build_resnet50_basic, build_efficientnet_v2_basic
from binary_classifier.train import setup_training, evaluate, train

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Locate the YAML file relative to the notebook's location
notebook_dir = os.path.dirname(os.path.abspath("__file__"))

# You need to update this path to your new .yaml file
config_path = os.path.join(notebook_dir, "../configs/default_single_ensemble_training.yaml")

# Load the YAML file
with open(config_path, "r") as f:
    config = yaml.safe_load(f)
print(config)

{'device': 'cuda', 'model': {'num_classes': 2, 'architecture': 'efficientnet', 'freeze_backbone': False, 'dropout': 0.2, 'hidden_units1': 100}, 'train': {'batch_size': 16, 'epochs': 3, 'lr': 0.001, 'momentum': 0.9, 'optimizer': 'sgd', 'criterion': 'cross_entropy'}, 'experiment': {'seed': 42, 'experiment_name': 'efficientnet_single_classifier'}, 'transforms': {'resize': [480, 480], 'horizontal_flip': 0.5, 'rotate': 15, 'jitter': {'brightness': 0.2, 'contrast': 0.2, 'saturation': 0.2, 'hue': 0.1}, 'custom': {'block_timestamp': True}}, 'log': {'img_count': 50}}


In [3]:
print(torch.__version__)
print(torch.backends.mps.is_available())
device = config["device"]
print(f"Running on device: {device}")

2.5.1
False
Running on device: cuda


In [4]:
train_features, test_features, train_labels, species_labels = load_data()
species_labels

['antelope_duiker',
 'bird',
 'blank',
 'civet_genet',
 'hog',
 'leopard',
 'monkey_prosimian',
 'rodent']

In [5]:
train_labels_antelope_duiker = train_labels["antelope_duiker"]
train_labels_bird = train_labels["bird"]
train_labels_blank = train_labels["blank"]
train_labels_civet_genet = train_labels["civet_genet"]
train_labels_hog = train_labels["hog"]
train_labels_leopard = train_labels["leopard"]
train_labels_monkey_prosimian = train_labels["monkey_prosimian"]
train_labels_rodent = train_labels["rodent"]

In [6]:
# Get transforms
train_transforms, val_transforms = get_transforms(config)

In [7]:
set_seeds(config["experiment"]["seed"])

X_train_antelope_duiker, X_val_antelope_duiker, y_train_antelope_duiker, y_val_antelope_duiker = split_data(train_features, train_labels_antelope_duiker)

X_train_bird, X_val_bird, y_train_bird, y_val_bird = split_data(train_features, train_labels_bird, type='sites')

X_train_blank, X_val_blank, y_train_blank, y_val_blank = split_data(train_features, train_labels_blank, type='sites')

X_train_civet_genet, X_val_civet_genet, y_train_civet_genet, y_val_civet_genet = split_data(train_features, train_labels_civet_genet, type='sites')

X_train_hog, X_val_hog, y_train_hog, y_val_hog = split_data(train_features, train_labels_hog, type='sites')

X_train_leopard, X_val_leopard, y_train_leopard, y_val_leopard = split_data(train_features, train_labels_leopard, type='sites')

X_train_monkey_prosimian, X_val_monkey_prosimian, y_train_monkey_prosimian, y_val_monkey_prosimian = split_data(train_features, train_labels_monkey_prosimian, type='sites')

X_train_rodent, X_val_rodent, y_train_rodent, y_val_rodent = split_data(train_features, train_labels_rodent, type='sites')

In [8]:
set_seeds(config["experiment"]["seed"])

# Create datasets antelope
train_dataset_antelope_duiker = ImagesDataset(
    features=X_train_antelope_duiker[0:100], 
    labels=y_train_antelope_duiker[0:100], 
    transform=train_transforms, 
    device=device)

val_dataset_antelope_duiker = ImagesDataset(
    features=X_val_antelope_duiker[0:100], 
    labels=y_val_antelope_duiker[0:100], 
    transform=val_transforms, 
    device=device)

# Create DataLoaders
train_loader_antelope_duiker = DataLoader(
    train_dataset_antelope_duiker, 
    batch_size=config["train"]["batch_size"], 
    shuffle=True, 
    #num_workers=4,
    pin_memory=False)

val_loader_antelope_duiker = DataLoader(
    val_dataset_antelope_duiker, 
    batch_size=config["train"]["batch_size"], 
    shuffle=False, 
    #num_workers=4,
    pin_memory=False)

model_antelope_duiker = build_efficientnet_v2_basic(
    num_classes = config["model"]["num_classes"],
    hidden_units1 = config["model"]["hidden_units1"],
    dropout = config["model"]["dropout"] 
)
model_antelope_duiker = model_antelope_duiker.to(device)

In [ ]:
# Create datasets bird
train_dataset_bird = ImagesDataset(
    features=X_train_bird,
    labels=y_train_bird,, 
    transform=train_transforms, 
    device=device)
val_dataset_bird, = ImagesDataset(
    features=X_val_bird, 
    labels=y_val_bird, 
    transform=val_transforms, 
    device=device)

# Create DataLoaders
train_loader_bird = DataLoader(
    train_dataset_bird, 
    batch_size=config["train"]["batch_size"], 
    shuffle=True, 
    pin_memory=True)

val_loader_bird = DataLoader(
    val_dataset_bird, 
    batch_size=config["train"]["batch_size"], 
    shuffle=False, 
    pin_memory=True)

model_bird = build_efficientnet_v2_basic(
    num_classes = config["model"]["num_classes"],
    hidden_units1 = config["model"]["hidden_units1"],
    dropout = config["model"]["dropout"] 
)
model_bird = model_bird.to(device)

In [ ]:
# Create datasets blank
train_dataset_blank = ImagesDataset(
    features=X_train_blank, 
    labels=y_train_blank, 
    transform=train_transforms, 
    device=device)
val_dataset_blank = ImagesDataset(
    features=X_val_blank, 
    labels=y_val_blank, 
    transform=val_transforms, 
    device=device)

# Create DataLoaders
train_loader_blank = DataLoader(
    train_dataset_blank, 
    batch_size=config["train"]["batch_size"], 
    shuffle=True, 
    pin_memory=True)

val_loader_blank = DataLoader(
    val_dataset_blank, 
    batch_size=config["train"]["batch_size"], 
    shuffle=False, 
    pin_memory=True)

model_blank = build_efficientnet_v2_basic(
    num_classes = config["model"]["num_classes"],
    hidden_units1 = config["model"]["hidden_units1"],
    dropout = config["model"]["dropout"] 
)
model_blank = model_blank.to(device)

In [ ]:
# Create datasets civet genet
train_dataset_civet_genet = ImagesDataset(
    features=X_train_civet_genet, 
    labels=y_train_civet_genet, 
    transform=train_transforms, 
    device=device)
val_dataset_civet_genet = ImagesDataset(
    features=X_val_civet_genet, 
    labels=y_val_civet_genet, 
    transform=val_transforms, 
    device=device)

# Create DataLoaders
train_loader_civet_genet = DataLoader(
    train_dataset_civet_genet, 
    batch_size=config["train"]["batch_size"], 
    shuffle=True, 
    pin_memory=True)

val_loader_civet_genet = DataLoader(
    val_dataset_civet_genet, 
    batch_size=config["train"]["batch_size"], 
    shuffle=False, 
    pin_memory=True)

model_civet_genet = build_efficientnet_v2_basic(
    num_classes = config["model"]["num_classes"],
    hidden_units1 = config["model"]["hidden_units1"],
    dropout = config["model"]["dropout"] 
)
model_civet_genet = model_civet_genet.to(device)

In [ ]:
# Create datasets hog
train_dataset_hog = ImagesDataset(
    features=X_train_hog, 
    labels=y_train_hog, 
    transform=train_transforms, 
    device=device)
val_dataset_hog = ImagesDataset(
    features=X_val_hog, 
    labels=y_val_hog, 
    transform=val_transforms, 
    device=device)

# Create DataLoaders
train_loader_hog = DataLoader(
    train_dataset_hog, 
    batch_size=config["train"]["batch_size"], 
    shuffle=True, 
    pin_memory=True)

val_loader_hog = DataLoader(
    val_dataset_hog, 
    batch_size=config["train"]["batch_size"], 
    shuffle=False, 
    pin_memory=True)


model_hog = build_efficientnet_v2_basic(
    num_classes = config["model"]["num_classes"],
    hidden_units1 = config["model"]["hidden_units1"],
    dropout = config["model"]["dropout"] 
)
model_hog = model_hog.to(device)

In [ ]:
# Create datasets leopard
train_dataset_leopard = ImagesDataset(
    features=X_train_leopard, 
    labels=y_train_leopard, 
    transform=train_transforms, 
    device=device)
val_dataset_leopard = ImagesDataset(
    features=X_val_leopard, 
    labels=y_val_leopard, 
    transform=val_transforms, 
    device=device)

# Create DataLoaders
train_loader_leopard = DataLoader(
    train_dataset_leopard, 
    batch_size=config["train"]["batch_size"], 
    shuffle=True, 
    pin_memory=True)

val_loader_leopard = DataLoader(
    val_dataset_leopard, 
    batch_size=config["train"]["batch_size"], 
    shuffle=False, 
    pin_memory=True)


model_leopard = build_efficientnet_v2_basic(
    num_classes = config["model"]["num_classes"],
    hidden_units1 = config["model"]["hidden_units1"],
    dropout = config["model"]["dropout"] 
)
model_leopard = model_leopard.to(device)

In [ ]:
# Create datasets monkey prosimian
train_dataset_monkey_prosimian = ImagesDataset(
    features=X_train_monkey_prosimian, 
    labels=y_train_monkey_prosimian, 
    transform=train_transforms, 
    device=device)
val_dataset_monkey_prosimian = ImagesDataset(
    features=X_val_monkey_prosimian, 
    labels=y_val_monkey_prosimian, 
    transform=val_transforms, 
    device=device)

# Create DataLoaders
train_loader_monkey_prosimian = DataLoader(
    train_dataset_monkey_prosimian, 
    batch_size=config["train"]["batch_size"], 
    shuffle=True, 
    pin_memory=True)

val_loader_monkey_prosimian = DataLoader(
    val_dataset_monkey_prosimian, 
    batch_size=config["train"]["batch_size"], 
    shuffle=False, 
    pin_memory=True)


model_monkey_prosimian = build_efficientnet_v2_basic(
    num_classes = config["model"]["num_classes"],
    hidden_units1 = config["model"]["hidden_units1"],
    dropout = config["model"]["dropout"] 
)
model_monkey_prosimian = model_monkey_prosimian.to(device)

In [ ]:
# Create datasets rodent
train_dataset_rodent = ImagesDataset(
    features=X_train_rodent, 
    labels=y_train_rodent, 
    transform=train_transforms, 
    device=device)
val_dataset_rodent = ImagesDataset(
    features=X_val_rodent, 
    labels=y_val_rodent, 
    transform=val_transforms, 
    device=device)

# Create DataLoaders
train_loader_rodent = DataLoader(
    train_dataset_rodent, 
    batch_size=config["train"]["batch_size"], 
    shuffle=True, 
    pin_memory=True)

val_loader = DataLoader(
    val_dataset_rodent, 
    batch_size=config["train"]["batch_size"], 
    shuffle=False, 
    pin_memory=True)



model_rodent = build_efficientnet_v2_basic(
    num_classes = config["model"]["num_classes"],
    hidden_units1 = config["model"]["hidden_units1"],
    dropout = config["model"]["dropout"] 
)
model_rodent = model_rodent.to(device)

In [9]:
model = model_antelope_duiker
train_loader = train_loader_antelope_duiker
val_loader = val_loader_antelope_duiker

In [10]:
wandb.require()
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: kseeger7 (kseeger7-georgia-institute-of-technology). Use `wandb login --relogin` to force relogin


True

In [11]:
# ✨ W&B: Initialize a new run to track this model's training
wandb.init(project="wildlife", config=config)

In [12]:
set_seeds(config["experiment"]['seed'])
criterion, optimizer = setup_training(
        model, 
        criterion=config["train"]["criterion"],
        optimizer=config["train"]["optimizer"], 
        lr=config["train"]["lr"], 
        momentum=config["train"]["momentum"])

In [13]:
log_counter = 0
tracking_loss_all = []
train_losses = []  # To store average training loss per epoch
val_losses = []    # To store validation loss per epoch
set_seeds(config["experiment"]['seed'])

start_time = time.time()
for epoch in range(config["train"]["epochs"]):
    # Training step
    avg_train_loss, tracking_loss = train(model, 
                                     train_loader, 
                                     criterion, 
                                     optimizer, 
                                     epoch, config, device=device)
    tracking_loss_all.extend(tracking_loss)  # Append to global list
    train_losses.append(avg_train_loss)  # Store avg training loss
    print(f"Epoch {epoch+1}/{config['train']['epochs']} - Avg Train Loss: {avg_train_loss:.4f}")
    
    # Evaluation step
    eval_metrics = evaluate(model, val_loader, criterion, config, epoch= epoch+1, device=device)
    val_losses.append(eval_metrics["loss"])  # Store validation loss
    print(f"Epoch {epoch+1}/{config['train']['epochs']} - Eval Loss: {eval_metrics['loss']:.4f}, Eval Acc: {eval_metrics['accuracy']:.2f}%")

end_time = time.time()
duration = end_time - start_time
wandb.log({"duration": duration})

Starting training for epoch 1


RuntimeError: expected scalar type Float but found Long

In [ ]:
# ✨ W&B: Mark the run as complete (Or wait until the end of notebook)
wandb.finish()